# Step 1: Data preprocessing and importing (Getting started)

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

we'll start off by the following steps:
- importing necessary libraries
- The data is messy, contain a lot of typos, unnecessary words, punctuations, etc, so we'll clean it using regex

In [3]:
!pip install rapidfuzz

In [4]:
import pandas as pd
import numpy as np
#nlp toolkit library helps us in giving us a list of stopwords that we use in cleaning data.
import nltk
from nltk.corpus import stopwords
import re
import random
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rapidfuzz import fuzz
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import pickle
from sklearn.metrics import classification_report
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor

In [5]:
df = pd.read_excel('/kaggle/input/product-matching-dataset/Product Matching Dataset.xlsx', sheet_name='Dataset')
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,sku,marketplace_product_name_ar,seller_item_name,price
0,854,تريتاس 5 مجم 14 قرص,تريتاس 5مجم 14قرص,57.0
1,3545,فيافاج 100 مجم 10 قرص,فيافاج 100مجم اقراص**,55.0
2,4158,كوديلاترول 30 قرص,كوديلاترول 3 شريط اقراص,52.5
3,4213,اليفو 5 مجم 20 قرص,اليفو اقراص,52.0
4,96,جابتن 400 مجم 30 كبسولة,جابتين400اقــــــراص جديد,156.0


In [6]:
nltk.download('stopwords')
stop_words = stopwords.words('arabic')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
stop_words.append('سج')
stop_words.append('سق')
stop_words.append('سعر')
stop_words.append('جديد')
stop_words.append('قديم')
stop_words.append('هام')
stop_words.append('العامريه')
stop_words.append('ادويا')
stop_words.append('ادويه')
stop_words.append('ركز')

In [8]:
def data_preprocessing(text):
  text = text.replace("أ", "ا")
  text = text.replace("آ", "ا")
  text = text.replace("إ", "ا")
  text = text.replace("ؤ", "و")
  text = text.replace("ئ", "ي")
  text = text.replace("ى", "ي")
  text = text.replace("ة", "ه")

  text = re.sub(r'[^\w\s]', ' ', text)
  text = re.sub(r'(\d*|\s*)جنيه$', ' ', text)
  text = re.sub(r'س\d*ج', " ", text)
  text = re.sub(r'س\d*ق', " ", text)
  text = re.sub(r"(\w)\1+", r'\1', text)
  text = re.sub(r"[ء-ي]*(ركيرل|ركريل)[ء-ي]*", " ", text)
  text = re.sub(r"\d+$", " ", text)
  text = re.sub(r"(\s)\1+", r"\1", text)
  text = text.strip()

  text = ' '.join([word for word in text.split() if word not in stop_words])
  return text



In [9]:
df['marketplace_product_name_ar'] = df['marketplace_product_name_ar'].apply(data_preprocessing)
df['seller_item_name'] = df['seller_item_name'].apply(data_preprocessing)
df

,sku,marketplace_product_name_ar,seller_item_name,price
0,854,تريتاس 5 مجم 14 قرص,تريتاس 5مجم 14قرص,57.0
1,3545,فيافاج 10 مجم 10 قرص,فيافاج 10مجم اقراص,55.0
2,4158,كوديلاترول 30 قرص,كوديلاترول 3 شريط اقراص,52.5
3,4213,اليفو 5 مجم 20 قرص,اليفو اقراص,52.0
4,96,جابتن 40 مجم 30 كبسوله,جابتين40اقـراص,156.0
...,...,...,...,...
83557,1134,تيرز جارد 0 3 قطره 15 مل,تيرز جارد قطره,30.0
83558,4139,البندازول 40 مجم 6 قرص,البندازول اقراص فارماكيور,46.0
83559,762,اوتريفين بيبي سالين نقط انف 15 مل,اوتريفين سالين,13.0
83560,446,فليكتور 50 مجم 30 كبسول,فليكتور 3 شريط,87.0


In [10]:
df_unique = df.drop_duplicates()
df_unique

,sku,marketplace_product_name_ar,seller_item_name,price
0,854,تريتاس 5 مجم 14 قرص,تريتاس 5مجم 14قرص,57.0
1,3545,فيافاج 10 مجم 10 قرص,فيافاج 10مجم اقراص,55.0
2,4158,كوديلاترول 30 قرص,كوديلاترول 3 شريط اقراص,52.5
3,4213,اليفو 5 مجم 20 قرص,اليفو اقراص,52.0
4,96,جابتن 40 مجم 30 كبسوله,جابتين40اقـراص,156.0
...,...,...,...,...
83548,30,كتافلام 50 مجم 20 قرص,كتافلام 50,86.0
83553,2869,لاميكتال 25 مجم 30 قرص,لامكتال 25مجم جلاكسو,54.0
83558,4139,البندازول 40 مجم 6 قرص,البندازول اقراص فارماكيور,46.0
83560,446,فليكتور 50 مجم 30 كبسول,فليكتور 3 شريط,87.0


In [11]:
df_unique['label'] = 1
df_unique

<ipython-input-11-a2e8327959d8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['label'] = 1


,sku,marketplace_product_name_ar,seller_item_name,price,label
0,854,تريتاس 5 مجم 14 قرص,تريتاس 5مجم 14قرص,57.0,1
1,3545,فيافاج 10 مجم 10 قرص,فيافاج 10مجم اقراص,55.0,1
2,4158,كوديلاترول 30 قرص,كوديلاترول 3 شريط اقراص,52.5,1
3,4213,اليفو 5 مجم 20 قرص,اليفو اقراص,52.0,1
4,96,جابتن 40 مجم 30 كبسوله,جابتين40اقـراص,156.0,1
...,...,...,...,...,...
83548,30,كتافلام 50 مجم 20 قرص,كتافلام 50,86.0,1
83553,2869,لاميكتال 25 مجم 30 قرص,لامكتال 25مجم جلاكسو,54.0,1
83558,4139,البندازول 40 مجم 6 قرص,البندازول اقراص فارماكيور,46.0,1
83560,446,فليكتور 50 مجم 30 كبسول,فليكتور 3 شريط,87.0,1


we'll put label 1 for the correctly matched drugs, and 0 for the unmatched, the data doesn't contain unmatched drugs, so we'll generate 5 unmatched examples for each drug.
we'll do this so the model can learn correctly the examples that indicate unmatch, and the examples that indicate a match.

In [12]:
df_master_file = pd.read_excel('/kaggle/input/product-matching-dataset/Product Matching Dataset.xlsx', sheet_name='Master File')
df_master_file.head()

,sku,product_name,product_name_ar,price
0,279,ANAFRONIL 75 MG 20 TAB,انافرونيل 75 مجم اس ار 20 قرص,75.0
1,2282,LOPRECOUGH SYRUP 100 ML,لوبريكاف شراب 100 مل,28.5
2,4331,TOMEX PLUS 50 TAB,تومكس بلس 50 قرص,60.0
3,1022,TAROLIMUS 0.03% OINT. 15 GM,تاروليمس 0.03 % مرهم 15 جم,129.0
4,116,GLIPTUS PLUS 50/1000 MG 30 TAB,جليبتس بلس 50/1000 مجم 30 قرص,192.0


In [13]:
def generate_negative_pairs(master, DF):
    #iterate over dataset df
    #for each sku (medicine), generate negative pairs for it (negative matches)
    arr = defaultdict(list)
    for idx in DF['sku']:
        #generate for example 5 negative matches, being put after the positive pairs,
        #the negative pairs can be (different medicine from master file: different sku, different sku but has high similarity)
        i = 0
        while i < 5:
            choice = random.choice(DF['sku'].unique())
            while choice == idx:
                choice = random.choice(DF['sku'].unique())
            drug_b = random.choice(DF.loc[DF['sku']==choice, 'seller_item_name'].to_list())
            arr[idx].append(drug_b)
            i += 1
    return arr

neg_pairs = generate_negative_pairs(df_master_file, df_unique)

In [14]:
new_rows = []

# Iterate over negative_pairs to create new rows
for sku, negatives in neg_pairs.items():
    # Get the correct marketplace product name for this SKU
    correct_name = df.loc[df['sku'] == sku, 'marketplace_product_name_ar'].values[0]

    # Create new rows for each negative pair
    for negative_drug in negatives:
        new_rows.append({'sku': sku, 
                         'marketplace_product_name_ar': correct_name, 
                         'seller_item_name': negative_drug, 
                         'label': 0})

# Convert new_rows into a DataFrame
negative_df = pd.DataFrame(new_rows)
negative_df.head()

,sku,marketplace_product_name_ar,seller_item_name,label
0,854,تريتاس 5 مجم 14 قرص,اليربان 1مجم 5مل شراب 10 مل,0
1,854,تريتاس 5 مجم 14 قرص,كتافلام 6 امبول 83,0
2,854,تريتاس 5 مجم 14 قرص,زيثروكان 3 كبسول,0
3,854,تريتاس 5 مجم 14 قرص,بيونال كبسوال,0
4,854,تريتاس 5 مجم 14 قرص,اميبريد 50 مجم اقراص 1ج,0


In [15]:
final_df = pd.concat([df_unique, negative_df], ignore_index=True)
final_df

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,sku,marketplace_product_name_ar,seller_item_name,price,label
0,854,تريتاس 5 مجم 14 قرص,تريتاس 5مجم 14قرص,57.0,1
1,3545,فيافاج 10 مجم 10 قرص,فيافاج 10مجم اقراص,55.0,1
2,4158,كوديلاترول 30 قرص,كوديلاترول 3 شريط اقراص,52.5,1
3,4213,اليفو 5 مجم 20 قرص,اليفو اقراص,52.0,1
4,96,جابتن 40 مجم 30 كبسوله,جابتين40اقـراص,156.0,1
...,...,...,...,...,...
167605,796,فنتال بخاخ انف 10 مكجم 20 جرعه,مايودورا 10مجم 30قرص,NaN,0
167606,796,فنتال بخاخ انف 10 مكجم 20 جرعه,امباكوزا 10مجم اقراص,NaN,0
167607,796,فنتال بخاخ انف 10 مكجم 20 جرعه,سيلدافا10 مجم,NaN,0
167608,796,فنتال بخاخ انف 10 مكجم 20 جرعه,تارجوفلوكساسين 750 2 شريط,NaN,0


# Step 2: feature extraction and model training
we'll do the following in this step:
- convert the text data into numerical data using tf-idf vectorizer
- create the similarity features that we'll pass to the model as training
- splitting the data and train and evaluate the model

In [16]:
# TF-IDF vectorizer
tfidf = TfidfVectorizer(analyzer='char_wb', ngram_range=(2,4))
all_names = pd.concat([final_df['marketplace_product_name_ar'], final_df['seller_item_name']])
tfidf.fit(all_names)

def compute_features(row):
    name1, name2 = row['marketplace_product_name_ar'], row['seller_item_name']
    vec1 = tfidf.transform([name1])
    vec2 = tfidf.transform([name2])
    
    cosine_sim = cosine_similarity(vec1, vec2)[0][0]  
    levenshtein = fuzz.ratio(name1, name2) / 100  
    jaro_winkler = fuzz.WRatio(name1, name2) / 100

    return pd.Series([cosine_sim, levenshtein, jaro_winkler])

final_df[['cosine_sim', 'levenshtein', 'jaro_winkler']] = final_df.apply(compute_features, axis=1)
final_df.head()

,sku,marketplace_product_name_ar,seller_item_name,price,label,cosine_sim,levenshtein,jaro_winkler
0,854,تريتاس 5 مجم 14 قرص,تريتاس 5مجم 14قرص,57.0,1,0.683875,0.944444,0.944444
1,3545,فيافاج 10 مجم 10 قرص,فيافاج 10مجم اقراص,55.0,1,0.779010,0.842105,0.842105
2,4158,كوديلاترول 30 قرص,كوديلاترول 3 شريط اقراص,52.5,1,0.799640,0.800000,0.800000
3,4213,اليفو 5 مجم 20 قرص,اليفو اقراص,52.0,1,0.769048,0.620690,0.855000
4,96,جابتن 40 مجم 30 كبسوله,جابتين40اقـراص,156.0,1,0.209827,0.388889,0.572727


In [17]:
X = final_df[['cosine_sim', 'levenshtein', 'jaro_winkler']]
y = final_df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Train XGBoost
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [18]:
y_pred = xgb_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     35130
           1       0.97      0.95      0.96      6773

    accuracy                           0.99     41903
   macro avg       0.98      0.97      0.98     41903
weighted avg       0.99      0.99      0.99     41903



the drugs that has multiple variations (multiple concentrations), we'll consider the output of any of them will be Not sure, so a person could manually check and review them, beceause the model might easily predict a wrong variation of the drug because of how similar and close they are.

In [19]:
medicine_frequency = defaultdict(int)
for full_drug in df_master_file['product_name']:
    drug = re.split(r'\d+', full_drug)
    medicine_frequency[drug[0]]+=1
    
for full_drug in df_master_file['product_name_ar']:
    drug = re.split(r'\d+', full_drug)
    medicine_frequency[drug[0]]+=1

# medicine_frequency

In [20]:
def find_best_match(input_name, known_names, model, tfidf):
    similarities = []
    features_list = []
    if 97 <= ord(input_name[0].lower()) <= 122:
        for known_name in known_names['product_name']:
            vec1 = tfidf.transform([input_name])
            vec2 = tfidf.transform([known_name])
    
            cosine_sim = cosine_similarity(vec1, vec2)[0][0]
            levenshtein = fuzz.ratio(input_name, known_name) / 100
            jaro_winkler = fuzz.WRatio(input_name, known_name) / 100
            
            features = np.array([[cosine_sim, levenshtein, jaro_winkler]])
            prob = model.predict_proba(features)[0][1]  # Match probability
            similarities.append((known_name, prob))
            
    else:
        for known_name in known_names['product_name_ar']:
            vec1 = tfidf.transform([input_name])
            vec2 = tfidf.transform([known_name])
            
            cosine_sim = cosine_similarity(vec1, vec2)[0][0]
            levenshtein = fuzz.ratio(input_name, known_name) / 100
            jaro_winkler = fuzz.WRatio(input_name, known_name) / 100
        
            features = np.array([[cosine_sim, levenshtein, jaro_winkler]])
            prob = model.predict_proba(features)[0][1]  # Match probability
            similarities.append((known_name, prob))
    # Get the best match
    best_match = max(similarities, key=lambda x: x[1])
    if (best_match[1]) < 0.95 or medicine_frequency[re.split(r'\d+', best_match[0])[0]] > 1:
        return (best_match, 'Not sure')
    return (best_match, 'sure')

find_best_match('esthl3t MG', df_master_file, xgb_model, tfidf)


(('ESTOHALT 40 MG 14 CAP', 0.9944607), 'sure')

the "find_best_match()" function is used for finding the best match for a single medicine.

but what if we want to pass in a sheet of medicines and get the best match for every one of them?
the problem of this is that it'll take time if the sheet is long, so what can we do?
Here comes the power of "parallel programming", we'll use ThreadPools to find the best match for a number of medicines (4 for example) at the same time.

In [21]:
test_df = pd.read_excel('/kaggle/input/product-matching-dataset/Product Matching Dataset.xlsx', sheet_name='Dataset')
test_df = test_df.sample(frac=1).reset_index(drop=True)
test_df.head()

,sku,marketplace_product_name_ar,seller_item_name,price
0,4093,الزميندا 20 مجم 21 قرص,الزميندا 20مجم 21قرص,66.0
1,2856,ديلاترول 6.25 مجم 30 قرص,ديلاترول 6.25مجم - 30قرص,31.5
2,918,نانازوكسيد 100مجم/5مل شراب معلق 60 مل,نانازوكسيد شراب,39.0
3,320,ميوكوستا 100 مجم 20 قرص,ميوكوستا100مجم اقراص جديد,79.0
4,2350,اتورستات 20 مجم 14 قرص,اتورستات 20مجم 14قرص,64.0


In [22]:
lst = test_df['seller_item_name'][:10]

In [23]:
def find_best_matches(medicines, master, model, tfidf):
    results = []
    for medicine in medicines:
        match = (medicine, find_best_match(medicine, master, model, tfidf))
        print(match)
        results.append(match)
    return results

def find_best_matches_parallel(full_medicines, master, model, tfidf, n_jobs=4):
    chunks = np.array_split(full_medicines, n_jobs)
    
    with ThreadPoolExecutor(max_workers=n_jobs) as executor:
        try:
            results = list(executor.map(lambda chunk: find_best_matches(chunk, master, model, tfidf), chunks))
        except KeyboardInterrupt:
            print("Interrupted! Shutting down threads...")
            executor.shutdown(wait=False, cancel_futures=True)
        return [item for sublist in results for item in sublist]

results = find_best_matches_parallel(lst, df_master_file, xgb_model, tfidf)

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


('ميوكوستا100مجم اقراص جديد', (('ميوكوستا 100 مجم 20 قرص', 0.99683774), 'sure'))
('افيل امبولات', (('افيل 45.5 مجم/2مل 6 امبول عضل', 0.9881023), 'sure'))
('الزميندا 20مجم 21قرص ', (('الزميندا 20 مجم 21 قرص', 0.99963844), 'sure'))
('فاسكولار اقراص', (('فاسكولار 20 مجم 20 قرص', 0.97132146), 'sure'))
('جليبتس 850/50 اقراص', (('جليبتس بلس 50/1000 مجم 30 قرص', 0.99797803), 'Not sure'))
('اتورستات 20مجم 14قرص', (('اتورستات 40 مجم 14 قرص', 0.99982506), 'Not sure'))('ديلاترول 6.25مجم - 30قرص', (('ديلاترول 6.25 مجم 30 قرص', 0.9994381), 'Not sure'))

('شاى رويال كبير_س ج', (('شاى ريجيم رويال 25 فلتر', 0.9752137), 'sure'))
('رومارين  امبولات(س ج', (('رومارين 75 مج 3 امبول', 0.9557624), 'sure'))
('نانازوكسيد شراب', (('نانازوكسيد 100مجم/5مل شراب معلق 60 مل', 0.99946696), 'Not sure'))


# Step 3: Model Deployment
- we'll build a class that contains all the steps we did, just for the case of retraining on a new dataset or anything.
- we also saved the class object, which contains the model and other important attributes like: tfidf, medicine_frequency dictionary, which both of them are used in the find_best_match function.

In [24]:
class MedicineMatcher():
    def __init__(self, model, tfidf, data_for_training, master_file, medicine_frequency):
        self.model = model
        self.tfidf = tfidf
        self.data = data_for_training
        self.master = master_file
        self.m_freq = medicine_frequency

    def __generate_stop_words(self):
        nltk.download('stopwords')
        stop_words = stopwords.words('arabic')
        stop_words.append('سج')
        stop_words.append('سق')
        stop_words.append('سعر')
        stop_words.append('جديد')
        stop_words.append('قديم')
        stop_words.append('هام')
        stop_words.append('العامريه')
        stop_words.append('ادويا')
        stop_words.append('ادويه')
        stop_words.append('ركز')
        return stop_words

    def __data_cleaning(self, text):
        text = text.replace("أ", "ا")
        text = text.replace("آ", "ا")
        text = text.replace("إ", "ا")
        text = text.replace("ؤ", "و")
        text = text.replace("ئ", "ي")
        text = text.replace("ى", "ي")
        text = text.replace("ة", "ه")
        
        text = re.sub(r'[^\w\s]', ' ', text)
        text = re.sub(r'(\d*|\s*)جنيه$', ' ', text)
        text = re.sub(r'س\d*ج', " ", text)
        text = re.sub(r'س\d*ق', " ", text)
        text = re.sub(r"(\w)\1+", r'\1', text)
        text = re.sub(r"[ء-ي]*(ركيرل|ركريل)[ء-ي]*", " ", text)
        text = re.sub(r"\d+$", " ", text)
        text = re.sub(r"(\s)\1+", r"\1", text)
        text = text.strip()

        text = ' '.join([word for word in text.split() if word not in stop_words])
        return text


    def __generate_negative_pairs(self, DF):
        arr = defaultdict(list)
        for idx in DF['sku']:
            #generate for example 5 negative matches, being put after the positive pairs,
            #the negative pairs can be (different medicine from master file: different sku, different sku but has high similarity)
            i = 0
            while i < 5:
                choice = random.choice(DF['sku'].unique())
                while choice == idx:
                    choice = random.choice(DF['sku'].unique())
                drug_b = random.choice(DF.loc[DF['sku']==choice, 'seller_item_name'].to_list())
                arr[idx].append(drug_b)
                i += 1
                
        new_rows = []

        # Iterate over negative_pairs to create new rows
        for sku, negatives in arr.items():
            # Get the correct marketplace product name for this SKU
            correct_name = self.data.loc[self.data['sku'] == sku, 'marketplace_product_name_ar'].values[0]
        
            # Create new rows for each negative pair
            for negative_drug in negatives:
                new_rows.append({'sku': sku, 
                                 'marketplace_product_name_ar': correct_name, 
                                 'seller_item_name': negative_drug, 
                                 'label': 0})
        
        # Convert new_rows into a DataFrame
        negative_df = pd.DataFrame(new_rows)
        return negative_df

    def data_preprocessing(self):
        stop_words = self.__generate_stop_words()
        self.data['marketplace_product_name_ar'] = self.data['marketplace_product_name_ar'].apply(self.__data_cleaning)
        self.data['seller_item_name'] = self.data['seller_item_name'].apply(self.__data_cleaning)
        df_unique = self.data.drop_duplicates()
        df_unique['label'] = 1
        negative_df = self.__generate_negative_pairs(df_unique)
        self.data = pd.concat([df_unique, negative_df], ignore_index=True)



    def __compute_similarities(self, row):
        name1, name2 = row['marketplace_product_name_ar'], row['seller_item_name']
        vec1 = self.tfidf.transform([name1])
        vec2 = self.tfidf.transform([name2])

        cosine_sim = cosine_similarity(vec1, vec2)[0][0]
        levenshtein = fuzz.ratio(name1, name2) / 100
        jaro_winkler = fuzz.WRatio(name1, name2) / 100

        return pd.Series([cosine_sim, levenshtein, jaro_winkler])
        
    def __medicineFrequency(self):
        for full_drug in self.master['product_name']:
            drug = re.split(r'\d+', full_drug)
            self.m_freq[drug[0]] += 1

        for full_drug in self.master['product_name_ar']:
            drug = re.split(r'\d+', full_drug)
            self.m_freq[drug[0]] += 1

    def Fit(self):
        # data preparation step
        self.tfidf = TfidfVectorizer(analyzer='char_wb', ngram_range=(2,4))
        all_names = pd.concat([self.data['marketplace_product_name_ar'], self.data['seller_item_name']])
        self.tfidf.fit(all_names)
        self.data[['cosine_sim', 'levenshtein', 'jaro_winkler']] = self.data.apply(self.__compute_similarities, axis=1)
        self.__medicineFrequency()
        
        #training
        X = self.data[['cosine_sim', 'levenshtein', 'jaro_winkler']]
        y = self.data['label']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
        self.model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
        self.model.fit(X_train, y_train)

    def evaluate(self):
        y_pred = xgb_model.predict(X_test)
        print(classification_report(y_test, y_pred))

    def find_best_match(self, input_name):
        """
        input_name: medicine name to be matched
        """
        similarities = []
        features_list = []
        if 97 <= ord(input_name[0].lower()) <= 122:
            for known_name in self.master['product_name']:
                vec1 = self.tfidf.transform([input_name])
                vec2 = self.tfidf.transform([known_name])
        
                cosine_sim = cosine_similarity(vec1, vec2)[0][0]
                levenshtein = fuzz.ratio(input_name, known_name) / 100
                jaro_winkler = fuzz.WRatio(input_name, known_name) / 100
                
                features = np.array([[cosine_sim, levenshtein, jaro_winkler]])
                prob = self.model.predict_proba(features)[0][1]  # Match probability
                similarities.append((known_name, prob))
                
        else:
            for known_name in self.master['product_name_ar']:
                vec1 = self.tfidf.transform([input_name])
                vec2 = self.tfidf.transform([known_name])
                
                cosine_sim = cosine_similarity(vec1, vec2)[0][0]
                levenshtein = fuzz.ratio(input_name, known_name) / 100
                jaro_winkler = fuzz.WRatio(input_name, known_name) / 100
            
                features = np.array([[cosine_sim, levenshtein, jaro_winkler]])
                prob = self.model.predict_proba(features)[0][1]  # Match probability
                similarities.append((known_name, prob))
        # Get the best match
        best_match = max(similarities, key=lambda x: x[1])
        if (best_match[1]) < 0.95 or self.m_freq[re.split(r'\d+', best_match[0])[0]] > 1:
            return (best_match, 'Not sure')
        return (best_match, 'sure')

    
    def __find_best_matches(self, medicines):
        results = []
        for medicine in medicines:
            match = (medicine, self.find_best_match(medicine))
            print(match)
            results.append(match)
        return results
    
    def find_best_matches_parallel(self, full_medicines, n_jobs=4):
        """
        full_medicines: a list of medicine names that need to be matched
        n_jobs: number of jobs to be done in parallel using ThreadPooling, used in splitting
        the list into chunks so each core will handle a chunk
        """
        chunks = np.array_split(full_medicines, n_jobs)
        
        with ThreadPoolExecutor(max_workers=n_jobs) as executor:
            try:
                results = list(executor.map(lambda chunk: self.__find_best_matches(chunk), chunks))
            except KeyboardInterrupt:
                print("Interrupted! Shutting down threads...")
                executor.shutdown(wait=False, cancel_futures=True)
            return [item for sublist in results for item in sublist]


In [25]:
medicine_matcher = MedicineMatcher(xgb_model, tfidf, final_df, df_master_file, medicine_frequency)

In [26]:
medicine_matcher.find_best_match('brofen')

(('BRAVAMAX 200 MG 10 TAB', 0.99546236), 'sure')

In [27]:
# Load the saved class
with open("/kaggle/input/medicine-matcher/medicine_matcher.pkl", "rb") as f:
    matcher = pickle.load(f)

In [28]:
matcher.find_best_match('estoh3lt')

(('ESTOHALT 40 MG 14 CAP', 0.9995497), 'sure')